In [1]:
%%capture
!pip install 'git+https://github.com/itaysegev/stable-baselines3.git'

In [2]:
from stable_baselines3 import SACD
from stable_baselines3 import SAC

In [3]:
%%capture
!git clone https://github.com/itaysegev/citylearn_utils.git

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
%cd citylearn_utils

/content/citylearn_utils


In [5]:
%%capture
!pip install -r requirements.txt

In [6]:
# System operations
import os

# Date and time
from datetime import datetime

# type hinting
from typing import List, Mapping, Tuple

# Data visualization
import matplotlib.pyplot as plt
from simulation_results import plot_actions, plot_rewards, plot_simulation_summary

# Data manipulation
import numpy as np

# CityLearn
from citylearn.data import DataSet
from citylearn.citylearn import CityLearnEnv
from citylearn.reward_function import RewardFunction, SolarPenaltyReward
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

# Simulation setup
from simulation_setup import set_schema_buildings, set_schema_simulation_period, set_active_observations
from utils import CustomCallback, get_loader, SACDCallback
from custom_rewards import SACCustomReward


In [7]:
# set all plotted figures without margins
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
DATASET_NAME = 'citylearn_challenge_2022_phase_all'
schema = DataSet.get_schema(DATASET_NAME)

root_directory = schema['root_directory']

# change the suffix number in the next code line to a
# number between 1 and 17 to preview other buildings
building_name = 'Building_1'

filename = schema['buildings'][building_name]['energy_simulation']
filepath = os.path.join(root_directory, filename)

In [9]:
RANDOM_SEED = 0

In [10]:
# edit next code line to change number of buildings in simulation
BUILDING_COUNT = 2

 # edit next code line to change number of days in simulation
DAY_COUNT = 7

# edit next code line to change active observations in simulation
# NOTE: More active observations could mean longer trainer time.
ACTIVE_OBSERVATIONS = ['hour', 'day_type']

schema, buildings = set_schema_buildings(schema, BUILDING_COUNT, RANDOM_SEED)
schema, simulation_start_time_step, simulation_end_time_step =\
    set_schema_simulation_period(schema, DAY_COUNT, RANDOM_SEED)
schema = set_active_observations(schema, ACTIVE_OBSERVATIONS)

print('Selected buildings:', buildings)
print(
    f'Selected {DAY_COUNT}-day period time steps:',
    (simulation_start_time_step, simulation_end_time_step)
)
print(f'Active observations:', ACTIVE_OBSERVATIONS)

Selected buildings: ['Building_2', 'Building_7']
Selected 7-day period time steps: (7392, 7559)
Active observations: ['hour', 'day_type']


In [11]:

# -------------------- CUSTOMIZE ENVIRONMENT --------------------
# Include other observations if needed.
# NOTE: More active observations could mean longer trainer time.
your_active_observations = [
    'hour',
    # 'day_type'
]

# ------------------ SET AGENT HYPERPARAMETERS ------------------
your_agent_kwargs = {
    'learning_rate': 0.001,
    'buffer_size': 1000000,
    'learning_starts': 100,
    'batch_size': 256,
    'tau': 0.005,
    'gamma': 0.99,
    'train_freq': 1,
    'weights_vector': [1, 1],
    'policy_kwargs': {'n_reward_components': 2}

}

# --------------- SET NUMBER OF TRAINING EPISODES ---------------
your_episodes = 30

# --------------- DEFINE CUSTOM REWARD FUNCTION -----------------
class YourCustomReward(SACCustomReward, SolarPenaltyReward):
    def __init__(self, env: CityLearnEnv):
        r"""Initialize CustomReward.

        Parameters
        ----------
        env: Mapping[str, CityLearnEnv]
            CityLearn environment instance.
        """

        SolarPenaltyReward.__init__(self, env)
        SACCustomReward.__init__(self, env)

    def calculate(self) -> List[float]:
        r"""Returns reward for most recent action.

        <Provide a description for your custom reward>.

        Parameters
        ----------
        reward: List[float]
            Reward for transition to current timestep.
        """
        reward = [sum(SolarPenaltyReward.calculate(self)), sum(SACCustomReward.calculate(self))]
        return reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
def train_your_custom_sacd(
    agent_kwargs: dict, episodes: int, reward_function: RewardFunction,
    building_count: int, day_count: int, active_observations: List[str],
    random_seed: int, reference_envs: Mapping[str, CityLearnEnv] = None,
    show_figures: bool = None
) -> dict:
    """Trains a custom soft-actor critic (SACD) agent on a custom environment.

    Trains an SAC agent using a custom environment and agent hyperparamter
    setup and plots the key performance indicators (KPIs), actions and
    rewards from training and evaluating the agent.

    Parameters
    ----------
    agent_kwargs: dict
        Defines the hyperparameters used to initialize the SAC agent.
    episodes: int
        Number of episodes to train the agent for.
    reward_function: RewardFunction
        A base or custom reward function class.
    building_count: int
        Number of buildings to set as active in schema.
    day_count: int
        Number of simulation days.
    active_observations: List[str]
        Names of observations to set active to be passed to control agent.
    random_seed: int
        Seed for pseudo-random number generator.
    reference_envs: Mapping[str, CityLearnEnv], default: None
        Mapping of user-defined control agent names to environments
        the agents have been used to control.
    show_figures: bool, default: False
        Indicate if summary figures should be plotted at the end of
        evaluation.

    Returns
    -------
    result: dict
        Results from training the agent as well as some input variables
        for reference including the following value keys:

            * random_seed: int
            * env: CityLearnEnv
            * model: SAC
            * actions: List[float]
            * rewards: List[float]
            * agent_kwargs: dict
            * episodes: int
            * reward_function: RewardFunction
            * buildings: List[str]
            * simulation_start_time_step: int
            * simulation_end_time_step: int
            * active_observations: List[str]
            * train_start_timestamp: datetime
            * train_end_timestamp: datetime
    """

    # get schema
    schema = DataSet.get_schema('citylearn_challenge_2022_phase_all')

    # select buildings
    schema, buildings = set_schema_buildings(
        schema, building_count, random_seed
    )
    print('Selected buildings:', buildings)

    # select days
    schema, simulation_start_time_step, simulation_end_time_step =\
        set_schema_simulation_period(schema, day_count, random_seed)
    print(
        f'Selected {day_count}-day period time steps:',
        (simulation_start_time_step, simulation_end_time_step)
    )

    # set active observations
    schema = set_active_observations(schema, active_observations)
    print(f'Active observations:', active_observations)

    # initialize environment
    env = CityLearnEnv(schema, central_agent=True)
    sacr_env = CityLearnEnv(schema, central_agent=True)

    # set reward function
    env.reward_function = reward_function(env=env)
    sacr_env.reward_function = SACCustomReward(sacr_env)

    # wrap environment
    env = NormalizedObservationWrapper(env)
    env = StableBaselines3Wrapper(env)

    sacr_env = NormalizedObservationWrapper(sacr_env)
    sacr_env = StableBaselines3Wrapper(sacr_env)

    # initialize agent

    model = SACD('MultiPerspectivePolicy', env, **agent_kwargs, seed=random_seed)
    sacr_model = SAC(policy='MlpPolicy', env=sacr_env, seed=RANDOM_SEED)

    # initialize loader
    total_timesteps = episodes*(env.time_steps - 1)
    print('Number of episodes to train:', episodes)

    # initialize SAC loader
    sac_modr_loader = get_loader(max=total_timesteps)
    print('Train SAC agent...')
    display(sac_modr_loader)

    # train SAC agent
    sacr_callback = CustomCallback(env=sacr_env, loader=sac_modr_loader)
    sacr_model = sacr_model.learn(total_timesteps=total_timesteps,
                                  callback=sacr_callback)

    # Evaluate the trained SAC model
    observations = sacr_env.reset()
    sacr_actions_list = []

    while not sacr_env.done:
        actions, _ = sacr_model.predict(observations, deterministic=True)
        observations, _, _, _ = sacr_env.step(actions)
        sacr_actions_list.append(actions)

    fig = plot_actions(sacr_actions_list, 'SAC Actions', sacr_env)
    plt.show()
    reference_envs={'SAC': sacr_env}

    # initialize SACD loader
    loader = get_loader(max=total_timesteps)
    print('Train SACD agent...')
    display(loader)

    # initialize callback
    weights_vector = agent_kwargs['weights_vector']
    callback = SACDCallback(env=env, loader=loader, weights_vector=weights_vector)

    # train SACD agent
    train_start_timestamp = datetime.utcnow()
    model = model.learn(total_timesteps=total_timesteps, callback=callback)
    train_end_timestamp = datetime.utcnow()

    # evaluate SACD agent
    observations = env.reset()
    actions_list = []

    while not env.done:
        actions, _ = model.predict(observations, deterministic=True)
        observations, _, _, _ = env.step(actions)
        actions_list.append(actions)

    # get rewards
    rewards = callback.reward_history[:episodes]

    # plot summary and compare with other control results
    if show_figures is not None and show_figures:
        env_id = 'SACD'

        reference_envs = {env_id: env, **reference_envs}
        plot_simulation_summary(reference_envs)

        # plot actions
        plot_actions(actions_list, f'{env_id} Actions', env)

        # plot rewards
        _, ax = plt.subplots(1, 1, figsize=(5, 2))
        ax = plot_rewards(ax, rewards, f'{env_id} Rewards')
        plt.tight_layout()
        plt.show()

    else:
        pass

    return {
        'random_seed': random_seed,
        'env': env,
        'model': model,
        'actions': actions_list,
        'rewards': rewards,
        'agent_kwargs': agent_kwargs,
        'episodes': episodes,
        'reward_function': reward_function,
        'buildings': buildings,
        'simulation_start_time_step': simulation_start_time_step,
        'simulation_end_time_step': simulation_end_time_step,
        'active_observations': active_observations,
        'train_start_timestamp': train_start_timestamp,
        'train_end_timestamp': train_end_timestamp,
    }

In [ ]:
your_results = train_your_custom_sacd(
    agent_kwargs=your_agent_kwargs,
    episodes=your_episodes,
    reward_function=YourCustomReward,
    building_count=BUILDING_COUNT,
    day_count=DAY_COUNT,
    active_observations=your_active_observations,
    random_seed=RANDOM_SEED,
    show_figures=True,
)

Selected buildings: ['Building_2', 'Building_7']
Selected 7-day period time steps: (7392, 7559)
Active observations: ['hour']


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Number of episodes to train: 30
Train SAC agent...


IntProgress(value=0, description='Simulating:', max=5010, style=ProgressStyle(bar_color='maroon'))

/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
